# Model Server

In [1]:
#!python -m pip install v3io

In [2]:
# nuclio: ignore
import nuclio

In [3]:
%nuclio config kind="nuclio:serving"
%nuclio env MODEL_CLASS=ClassifierModel
%nuclio config spec.build.baseImage = "mlrun/mlrun"

%nuclio: setting kind to 'nuclio:serving'
%nuclio: setting 'MODEL_CLASS' environment variable
%nuclio: setting spec.build.baseImage to 'mlrun/ml-models'


In [4]:
%%nuclio cmd -c
python -m pip install numpy cloudpickle v3io sklearn

In [5]:
import os
from cloudpickle import load
import numpy as np
from typing import List
from datetime import datetime
import mlrun

In [7]:
class ClassifierModel(mlrun.runtimes.MLModelServer):
    def load(self):
        """Load model from KubeFlow storage.
        """
        """Load model from storage."""
        model_file, extra_data = self.get_model('.pkl')
        self.model = load(open(model_file, 'rb'))

    def predict(self, body: dict) -> List:
        """Generate model predictions from sample.
        
        :param body : A dict of observations, each of which is an 1-dimensional feature vector.
            
        Returns model predictions as a `List`, one for each row in the `body` input `List`.
        """
        start = datetime.now()
        try:
            feats = np.asarray(body['instances'])
            result: np.ndarray = self.model.predict(feats)
            resp = result.tolist()
        except Exception as e:
            raise Exception(f"Failed to predict {e}")
        
        return resp

In [8]:
# nuclio: end-code

## test models locally and deploy

The sklearn-project generated 3 models that will be deployed in the server project `sklearn-servers`

### test locally

In [9]:
import cloudpickle as cp
models_path = '/User/ml/demos/sklearn-pipe/models'

from sklearn.datasets import load_iris
iris = load_iris()

x = iris['data'].tolist()
y = iris['target']

for model in os.listdir(models_path):
    if model.endswith(".pkl"):
        
        my_server = ClassifierModel('classifier', model_dir=os.path.join(models_path, model))
        my_server.load()

        a = my_server.predict({"instances": x})

        assert len(a)==150